# Generate MEIs for one Dataset

In [1]:
%load_ext autoreload
%autoreload 2

import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config['schema_name'] = "nnfabrik_color_mei"
schema = dj.schema("nnfabrik_color_mei")

from matplotlib import pyplot as plt
from torch import load

from nnfabrik.main import *
import nnfabrik
from nnfabrik import main, builder
import os
from os import listdir
from os.path import isfile, join
import torch
import matplotlib.pyplot as plt
import numpy as np
from neuralpredictors.measures import corr

from nndichromacy.tables.from_nnfabrik import TrainedModel
from nndichromacy.tables.from_mei import MEI, MEIMethod, MEISeed, MEISelector, TrainedEnsembleModel

Connecting kwilleke@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


# Select Dataset

In [7]:
dataset_hash = 'd6048152f199d06e47cc7245a39bd671'
key = dict(dataset_hash=dataset_hash)

# Creating the ensemble

#### Make sure there are only 5 models trained on that dataset. If there are more, add restrictions

In [5]:
TrainedModel() & key

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion
nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,1000,standard_trainer.full_gauss_cortex.23656-27-8,0.234982,=BLOB=,kwilleke,2020-06-12 11:10:32
nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,2000,standard_trainer.full_gauss_cortex.23656-27-8,0.225349,=BLOB=,kwilleke,2020-06-12 11:13:39
nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,3000,standard_trainer.full_gauss_cortex.23656-27-8,0.233924,=BLOB=,kwilleke,2020-06-12 11:16:52
nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,4000,standard_trainer.full_gauss_cortex.23656-27-8,0.231656,=BLOB=,kwilleke,2020-06-12 11:20:23
nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,5000,standard_trainer.full_gauss_cortex.23656-27-8,0.225965,=BLOB=,kwilleke,2020-06-12 11:23:27


In [ ]:
# creating the ensemble
TrainedEnsembleModel().create_ensemble(key=key, comment='default')

In [12]:
# Check whether the ensemble has the correct amount of members:
TrainedEnsembleModel.Member() & key

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble,model_fn name of the model function,model_hash hash of the model configuration,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder
nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,f03c05a2673692335163e305915c0eae,nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,1000
nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,f03c05a2673692335163e305915c0eae,nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,2000
nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,f03c05a2673692335163e305915c0eae,nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,3000
nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,f03c05a2673692335163e305915c0eae,nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,4000
nndichromacy.datasets.static_loaders,d6048152f199d06e47cc7245a39bd671,f03c05a2673692335163e305915c0eae,nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,5000


# Populating the MEISelector table, to match the unit IDs and the units within the model

In [13]:
MEISelector().populate(key, display_progress=True)

  0%|          | 0/1 [00:00<?, ?it/s]

static23656-27-8-preproc0 exists already. Not unpacking static23656-27-8-preproc0.zip


100%|██████████| 1/1 [00:00<00:00,  2.14it/s]


# Selecting the MEIMethod

#### The default here is for normalized images

In [17]:
method_hash = 'e203277aa2540dd9f907344ed5babb21'

MEIMethod & dict(method_hash=method_hash)

method_fn name of the method function,method_hash hash of the method config,method_config method configuration object,method_comment a short comment describing the method,method_ts UTZ timestamp at time of insertion
featurevis.methods.gradient_ascent,e203277aa2540dd9f907344ed5babb21,=BLOB=,"std=0.1, gauss_blur=1, lr=1, iter=500",2020-05-30 11:02:19


# Populate the MEI table

In [ ]:
mei_key = dict(method_hash=method_hash, dataset_hash=dataset_hash)

# make sure that there is only one ensemble for the dataset
assert len(TrainedEnsembleModel & dict(dataset_hash=dataset_hash)) == 1, f"Multiple Ensembles found for dataset_hash: {dataset_hash}"

# if there are multiple ensembles for one dataset, then the ensemble_hash has to be specified in the key, for example:
# mei_key = dict(method_hash=method_hash, dataset_hash=dataset_hash, ensemble_hash=some_hash)

MEI.populate(mei_key, display_progress=True, order='random', reserve_jobs=True)

  0%|          | 0/1085 [00:00<?, ?it/s]

static23656-27-8-preproc0 exists already. Not unpacking static23656-27-8-preproc0.zip
static23656-27-8-preproc0 exists already. Not unpacking static23656-27-8-preproc0.zip
static23656-27-8-preproc0 exists already. Not unpacking static23656-27-8-preproc0.zip
static23656-27-8-preproc0 exists already. Not unpacking static23656-27-8-preproc0.zip
static23656-27-8-preproc0 exists already. Not unpacking static23656-27-8-preproc0.zip


  2%|▏         | 17/1085 [03:53<4:03:22, 13.67s/it]